In [418]:
import pandas as pd
import xlsxwriter
from datetime import datetime, timedelta

In [419]:
cols = ["Datetime","TIC-350.PV"]

df = pd.read_csv("T.csv",header=4,encoding='utf-8')
df = df.iloc[8:]
df["Datetime"] = df["Unnamed: 2"] +" "+ df["Unnamed: 3"]
df = df[cols]

In [420]:
#format datatypes to YYYY:MM:DD HH
df["Datetime"] = pd.to_datetime(df["Datetime"])
df["TIC-350.PV"] = pd.to_numeric(df["TIC-350.PV"])

df["Datetime"] = pd.to_datetime(df["Datetime"])
df["date_hours"] = df["Datetime"].dt.strftime('%Y-%m-%d %H')
df["date_hours"] = df["date_hours"].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H'))

strt_yr = df["Datetime"].iloc[0].year
strt_mo = df["Datetime"].iloc[0].month
strt_day = df["Datetime"].iloc[0].day
end_yr = df["Datetime"].iloc[-1].year
end_mo = df["Datetime"].iloc[-1].month
end_day = df["Datetime"].iloc[0].day

In [421]:
#T < 1400 minute events
less_1400 = df[df["TIC-350.PV"] < 1400]
dates = less_1400["Datetime"].dt.strftime('%Y-%m-%d')
times = less_1400["Datetime"].dt.strftime('%H:%M')
temp = less_1400["TIC-350.PV"].values

In [422]:
#calculate 3 hr averages for T < 1400 minute events
date_hours = less_1400["Datetime"].dt.strftime('%Y-%m-%d %H')
date_hours = date_hours.apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H'))

#break hourly continuous deviation events 
events = []
for x in date_hours:
    for i in range(-3,4):
        t = x + timedelta(hours=(i))
        if t not in events:
            events.append(t)
            
#convert events to date and hour strings
events_date = [x.strftime('%Y-%m-%d') for x in events]
events_hour = [x.strftime('%H') for x in events]

#calculate hourly avgs
hr_avgs = []
for x in events:
    mean = round(df[df["date_hours"] == x].mean(),1)
    hr_avgs.append(mean)

#calculate running three hour avg
three_hr_avgs = []
for x in events:
    mean = round(df[(df["date_hours"] >= (x+timedelta(hours=-2))) & (df["date_hours"] <= x)].mean(),1)
    three_hr_avgs.append(mean)
    
#generate status based on three hour avg
actions = []
for x in three_hr_avgs:
    if float(x) < 1400:
        actions.append("REPORTABLE")
    else:
        actions.append("No Action")

In [423]:
#setup workbook and worksheet
workbook = xlsxwriter.Workbook('%d_%d_%d to %d_%d_%d NESHAP 7A.xlsx' %(strt_yr,strt_mo,strt_day,end_yr,end_mo,end_day))
worksheet1 = workbook.add_worksheet("3_HR_Avgs")
worksheet2 = workbook.add_worksheet("Minutes, T < 1400")
bold = workbook.add_format({'bold':True})

worksheet1.set_column('A:A',15)
worksheet1.set_column('C:E',12)
worksheet2.set_column('A:A',15)


#write filtered data into cells
col = 0
row = 0

#write tab descriptions
worksheet1.write(1,8,"NESHAP 7A Requirements: Report if I-301 temp falls below 1400F on a running 3 hour average.")
worksheet2.write(1,8, "This tab shows minute data when I-301 temp fell below 1400F.")

#write headers
headers1 = ["Date","Hour","HR Avg (F)","3HR Avg","Action"]
for x in headers1:
    worksheet1.write(row,col,x,bold)
    col += 1
col = 0
headers2 = ["Date","Time","TIC-350.PV"]
for x in headers2:
    worksheet2.write(row,col,x,bold)
    col += 1

    
#write hr and 3hr avg data
col = 0
row = 1
lists1 = [events_date,events_hour,hr_avgs,three_hr_avgs,actions]
for x in lists1:
    for i in x:
        worksheet1.write(row,col,i)
        row += 1
    col += 1
    row = 1


    
#write minute data    
col = 0
row = 1
lists2 = [dates, times, temp]
for x in lists2:
    for i in x:
        worksheet2.write(row,col,i)
        row += 1
    col += 1
    row = 1

    
workbook.close()
